In [1]:
import requests
import json
import pandas as pd
import geopandas as gpd
import contextily as cx
import matplotlib.pyplot as plt
from netAPI import pickleLoad, pickleDump
import warnings
warnings.filterwarnings('ignore')

/home/mingyi/anaconda3/envs/geo38/lib/python3.8/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [7]:
def query_generator(bound):
    query = {'transportation':"""[out:json][timeout:30000];(
    node["amenity"~"bicycle_parking|bicycle_rental|boat_sharing|bus_station|car_rental|car_sharing|charging_station|ferry_terminal|fuel|motorcycle_parking|parking|parking_entrance|parking_space|taxi"](%s);
    way["amenity"~"bicycle_parking|bicycle_rental|boat_sharing|bus_station|car_rental|car_sharing|charging_station|ferry_terminal|fuel|motorcycle_parking|parking|parking_entrance|parking_space|taxi"](%s);
    relation["amenity"~"bicycle_parking|bicycle_rental|boat_sharing|bus_station|car_rental|car_sharing|charging_station|ferry_terminal|fuel|motorcycle_parking|parking|parking_entrance|parking_space|taxi"](%s);
    );
    out center;

    """%(bound,bound,bound),
    'restaurant':"""
    [out:json][timeout:30000];
    (
    node["amenity"~"bar|biergarten|cafe|fast_food|food_court|ice_cream|pub|restaurant"](%s);
    way["amenity"~"bar|biergarten|cafe|fast_food|food_court|ice_cream|pub|restaurant"](%s);
    relation["amenity"~"bar|biergarten|cafe|fast_food|food_court|ice_cream|pub|restaurant"](%s);
    );
    out center;

    """%(bound,bound,bound), 
    'health':
            """
    [out:json][timeout:30000];
    (
    node["amenity"~"baby_hatch|clinic|dentist|doctors|hospital|nursing_home|social_facility|veterinary"](%s);
    way["amenity"~"baby_hatch|clinic|dentist|doctors|hospital|nursing_home|social_facility|veterinary"](%s);
    relation["amenity"~"baby_hatch|clinic|dentist|doctors|hospital|nursing_home|social_facility|veterinary"](%s);
    );
    out center;

    """%(bound,bound,bound),

    'education':"""
    [out:json][timeout:30000];
    (
    node["amenity"~"college|driving_school|kindergarten|language_school|library|toy_library|research_institute|training|music_school|school|traffic_park|university"](%s);
    way["amenity"~"college|driving_school|kindergarten|language_school|library|toy_library|research_institute|training|music_school|school|traffic_park|university"](%s);
    relation["amenity"~"college|driving_school|kindergarten|language_school|library|toy_library|research_institute|training|music_school|school|traffic_park|university"](%s);
    );
    out center;

    """%(bound,bound,bound),
    'retail':"""
    [out:json][timeout:30000];
    (
    node[landuse=retail](%s);
    way[landuse=retail](%s);
    relation[landuse=retail](%s);
    );
    out center;

    """%(bound,bound,bound)
            }
    return query

In [8]:
def OSMdownload(ct_map,city):
    overpass_url = "http://overpass-api.de/api/interpreter"
    bound = ct_map.dissolve().bounds.values.tolist()[0]
    bound = [bound[1],bound[0],bound[3],bound[2]]
    bound = ','.join([str(i) for i in bound])
    print(bound)
    query = query_generator(bound)
    for amenity in query.keys():
        print(amenity)
        overpass_query = query[amenity]
        response = requests.get(overpass_url, 
                        params={'data': overpass_query})
        data = response.json()
        x = []
        y = []
#         name = []
        for element in data['elements']:

            if element['type'] == 'node':
                lon = element['lon']
                lat = element['lat']
                x.append(lon)
                y.append(lat)
#                 amenity.append(element['tags']['amenity'])
            elif 'center' in element:
                lon = element['center']['lon']
                lat = element['center']['lat']
                x.append(lon)
                y.append(lat)
#                 amenity.append(element['tags']['amenity'])

#         pd.DataFrame({'x':x,'y':y}).to_csv('LEHD/transportation.csv',index=False)
        df = pd.DataFrame({'x':x,'y':y})
        gdf = gpd.GeoDataFrame(df,geometry=gpd.points_from_xy(df.x, df.y), crs="EPSG:4326")
#         print(ct_map)
        ct_map = ct_map.merge(gdf.sjoin(ct_map,op='within',how='left').groupby(['GEOID']).count()['x'].\
        reset_index().rename(columns={'x':amenity}),on='GEOID',how='left')
        print(ct_map.shape)
    ct_map[['GEOID','transportation', 'restaurant',
       'health', 'education', 'retail']].fillna(0).to_csv('LEHD/%sPOI.csv'%city,index=False)
    return ct_map

In [178]:
path = 'LEHD/'

cities = [
    ('New York City', 'ny', ['New York County, NY', 'Queens County, NY','Kings County, NY','Bronx County, NY','Richmond County, NY']),
    ('Los Angeles', 'ca', ['Los Angeles County, CA']),
    ('Chicago', 'il', ['Cook County, IL']),
    ('Houston', 'tx', ['Harris County, TX']),
    ('Boston', 'ma', ['Suffolk County, MA', 'Middlesex County, MA']),
    ('Phoenix', 'az', ['Maricopa County, AZ']),
    ('Philadelphia', 'pa', ['Philadelphia County, PA']),
    ('San Antonio', 'tx', ['Bexar County, TX']),
    ('San Diego', 'ca', ['San Diego County, CA']),
    ('Dallas', 'tx', ['Dallas County, TX']),
    ('San Jose', 'ca', ['Santa Clara County, CA']),
    ('Austin', 'tx', ['Travis County, TX']),
]

for city,state, counties in cities:
    print(city,state,'processing LEHD')
    LEHD = pd.read_csv(f'LEHD/{city}_flow.csv')
#     LEHD.O = LEHD.O.astype(str)
#     LEHD.D = LEHD.D.astype(str)
    nodes = list(set(LEHD.O.unique().tolist()+LEHD.D.unique().tolist()))
    print(len(nodes))
    ct_map = gpd.read_file('LEHD/'+state+'.zip')
    ct_map.GEOID = ct_map.GEOID.astype(int)
    ct_map = ct_map.loc[ct_map.GEOID.isin(nodes)]
    ct_map = ct_map.to_crs('epsg:4326')
    poi = OSMdownload(ct_map,city)
#     break

New York City ny processing LEHD
2318
40.491167,-74.258754,40.917705,-73.700169
transportation
(2318, 14)
restaurant
(2318, 15)
health
(2318, 16)
education
(2318, 17)
retail
(2318, 18)
Los Angeles ca processing LEHD
2495
32.75004,-118.94518,34.823302,-117.646374
transportation
(2495, 14)
restaurant
(2495, 15)
health
(2495, 16)
education
(2495, 17)
retail
(2495, 18)
Chicago il processing LEHD
1331
41.469705,-88.26364099999999,42.154292,-87.52366099999999
transportation
(1331, 14)
restaurant
(1331, 15)
health
(1331, 16)
education
(1331, 17)
retail
(1331, 18)
Houston tx processing LEHD
1115
29.497339,-95.96073299999999,30.170606,-94.908492
transportation
(1115, 14)
restaurant
(1115, 15)
health
(1115, 16)
education
(1115, 17)
retail
(1115, 18)
Boston ma processing LEHD
591
42.156782,-71.89877299999999,42.736554,-70.920101
transportation
(591, 14)
restaurant
(591, 15)
health
(591, 16)
education
(591, 17)
retail
(591, 18)
Phoenix az processing LEHD
1009
32.505058999999996,-113.335075,34.0481